# Assignment CNNs - Part 2: Transfer Learning Strategies with Fast Food Classification


In this notebook, we will:
1. Import and prepare the Fast Food Classification dataset
2. Implement three different transfer learning strategies:
   - Feature extraction (frozen pre-trained model)
   - Fine-tuning last few layers
   - Full fine-tuning
3. Get experience on how sample size affects the strategy's performance
4. Get experience of the impact of choosing the right/wrong learning rate
5. Get experience with the different transfer learning strategies

## 0. Introduction

Transfer learning is a powerful technique that allows us to leverage pre-trained models for new tasks. However, different transfer learning strategies are suitable for different scenarios, particularly depending on:
- Sample size of the target dataset
- Similarity between source and target tasks
- Available computational resources

In this exercise, we'll explore different transfer learning strategies using the "Fast Food Classification Dataset - V2" from Kaggle.

## 1. Data Import and Cleaning

1. Download the ["Fast Food Classification Dataset - V2"](https://www.kaggle.com/datasets/utkarshsaxenadn/fast-food-classification-dataset/data) from Kaggle.

2. Unzip and ensure that the dataset in Colab is structured as follows:

```
|
|-- Fast Food Classification V2
|   |-- TFRecords
|   |-- Test
|   |-- Train
|   `-- Valid
|-- sample_data
`-- archive.zip
```

`Fast Food Classification V2` and `archive.zip` should be on the same level as the already existing `sample_data` directory from Colab. You can ignore the `TFRecords` subdirectory in the following tasks.

In [5]:
#!/bin/bash
!curl -L -o archive.zip https://www.kaggle.com/api/v1/datasets/download/utkarshsaxenadn/fast-food-classification-dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0  820M    0 80466    0     0   106k      0  2:11:13 --:--:--  2:11:13  106k
  1  820M    1 8589k    0     0  5135k      0  0:02:43  0:00:01  0:02:42 9083k
  2  820M    2 20.6M    0     0  7896k      0  0:01:46  0:00:02  0:01:44 10.5M
  4  820M    4 33.1M    0     0  9230k      0  0:01:31  0:00:03  0:01:28 11.2M
  5  820M    5 45.4M    0     0  9964k      0  0:01:24  0:00:04  0:01:20 11.5M
  7  820M    7 57.7M    0     0  10.1M      0  0:01:20  0:00:05  0:01:15 11.6M
  8  820M    8 66.2M    0     0  9792k      0  0:01:25  0:00:06  0:01:19 11.0M
  9  820M    9 75.4M    0     0   9.8M      0  0:01:23  0:00:07  0:01:16 10.9M
 10  820M   10 86.7M    0     0  10.0M      0  0:0

In [43]:
!unzip -q archive.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


3. Apply the provided `delete_invalid_images` function to check and remove any corrupted or invalid images in each of the three dataset directories.

In [44]:
import os
from PIL import Image

# Define the paths to the directories
directories = [
    "Fast Food Classification V2/Train", #TODO: Set the correct path
    "Fast Food Classification V2/Valid", #TODO: Set the correct path
    "Fast Food Classification V2/Test" #TODO: Set the correct path
]

# Define supported image file extensions
supported_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp'}

def delete_invalid_images(directory):
    """Goes through a directory and deletes any invalid or unsupported images."""
    print(f"Checking directory: {directory}")
    check_directory = os.path.exists(directory)
    if not check_directory:
        print(f"Directory does not exist: {directory}")
        return
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            # Check if the file has a supported extension
            if not any(file.lower().endswith(ext) for ext in supported_extensions):
                print(f"Deleting unsupported file: {file_path}")
                os.remove(file_path)
                continue

            try:
                with Image.open(file_path) as img:
                    img.load()  # Ensure the image content can be read
                    if img.format not in ["JPEG", "PNG", "GIF", "BMP"]:
                        raise IOError("Unsupported image format")
            except (IOError, SyntaxError, AttributeError) as e:
                print(f"Deleting invalid or corrupted image: {file_path}")
                os.remove(file_path)

# Run the function for each directory
for dir_path in directories:
    delete_invalid_images(dir_path)

print("Invalid and unsupported image cleanup completed.")

Checking directory: Fast Food Classification V2/Train
Deleting invalid or corrupted image: Fast Food Classification V2/Train\Hot Dog\Hot Dog - Train (59).jpg
Checking directory: Fast Food Classification V2/Valid
Checking directory: Fast Food Classification V2/Test
Invalid and unsupported image cleanup completed.


4. To reduce training time, only three classes are considered for the following training.

In [45]:
# prompt: In each of the Train, Test, and Valid folder in the folder "Fast Food Classification V2" that were just unzipped, only keep the folders Taquito, Crispy Chicken, and Donut. Remove the others.

import os
import shutil

# Define the root directory of the dataset
dataset_root = "Fast Food Classification V2"

# Define the subdirectories (Train, Test, Valid)
subdirectories = ["Train", "Test", "Valid"]

# Define the classes to keep
classes_to_keep = ["Taquito", "Crispy Chicken", "Donut"]

# Loop through each subdirectory
for subdir in subdirectories:
  subdir_path = os.path.join(dataset_root, subdir)

  # Loop through each folder (class) in the subdirectory
  for class_folder in os.listdir(subdir_path):
    class_folder_path = os.path.join(subdir_path, class_folder)

    # Check if it's a directory and if it's not one of the classes to keep
    if os.path.isdir(class_folder_path) and class_folder not in classes_to_keep:
      print(f"Removing folder: {class_folder_path}")
      shutil.rmtree(class_folder_path)

print("Folder cleanup completed.")

Removing folder: Fast Food Classification V2\Train\Baked Potato
Removing folder: Fast Food Classification V2\Train\Burger
Removing folder: Fast Food Classification V2\Train\Fries
Removing folder: Fast Food Classification V2\Train\Hot Dog
Removing folder: Fast Food Classification V2\Train\Pizza
Removing folder: Fast Food Classification V2\Train\Sandwich
Removing folder: Fast Food Classification V2\Train\Taco
Removing folder: Fast Food Classification V2\Test\Baked Potato
Removing folder: Fast Food Classification V2\Test\Burger
Removing folder: Fast Food Classification V2\Test\Fries
Removing folder: Fast Food Classification V2\Test\Hot Dog
Removing folder: Fast Food Classification V2\Test\Pizza
Removing folder: Fast Food Classification V2\Test\Sandwich
Removing folder: Fast Food Classification V2\Test\Taco
Removing folder: Fast Food Classification V2\Valid\Baked Potato
Removing folder: Fast Food Classification V2\Valid\Burger
Removing folder: Fast Food Classification V2\Valid\Fries
Removi

### Creation of Different Training Dataset Sizes

   - Small: 50 images per class
   - Medium: 200 images per class  
   - Full: All available training images

To reduce training time, only three classes are considered in the training.

The validation and test sets will remain constant to ensure fair comparison.

In [46]:
import os
import numpy as np
import shutil
from pathlib import Path

# Constants
SMALL_SAMPLES = 50  # images per class
MEDIUM_SAMPLES = 200 # images per class

def create_sample_dataset(source_dir, target_dir, samples_per_class):
    """Creates a smaller dataset by randomly sampling from source directory"""
    if os.path.exists(target_dir):
        shutil.rmtree(target_dir)

    # Create target directory
    os.makedirs(target_dir)

    # For each class directory
    for class_dir in os.listdir(source_dir):
        source_class_path = os.path.join(source_dir, class_dir)
        target_class_path = os.path.join(target_dir, class_dir)

        if os.path.isdir(source_class_path):
            # Create class directory in target
            os.makedirs(target_class_path)

            # Get list of all images
            images = os.listdir(source_class_path)

            # Randomly sample specified number of images
            selected_images = np.random.choice(
                images,
                size=min(samples_per_class, len(images)),
                replace=False
            )

            # Copy selected images
            for img in selected_images:
                shutil.copy2(
                    os.path.join(source_class_path, img),
                    os.path.join(target_class_path, img)
                )

# Create sampled datasets
create_sample_dataset("./Fast Food Classification V2/Train", "./Train_Small", SMALL_SAMPLES)
create_sample_dataset("./Fast Food Classification V2/Train", "./Train_Medium", MEDIUM_SAMPLES)

## 2. Data Preprocessing

1. Load the training, validation and test datasets using the `image_dataset_from_directory` function. Specifically, set appropriate values for `label_mode`, `batch_size`, `image_size` and `shuffle`. Each dataset should get a fixed `seed` of `42`.

Checkout the documentation for details:
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory



In [47]:
import tensorflow as tf

# Constants
LABEL_MODE = 'int'
SEED = 42
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
#Tip: Models that will be used with this data work usually best when the input images are of the same size as in the original training of the model.
BATCH_SIZE = 64

# Load the small dataset
train_data_small = tf.keras.preprocessing.image_dataset_from_directory(
    "Train_Small",
    label_mode=LABEL_MODE,
    seed=SEED,  # Any fixed value works for reproducibility
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle= None #TODO: Set appropriate value
)

# Load the medium dataset
train_data_medium = tf.keras.preprocessing.image_dataset_from_directory(
    "Train_Medium",
    label_mode=LABEL_MODE,
    seed=SEED,  # Any fixed value works for reproducibility
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=None #TODO: Set appropriate value
)

# Load the full dataset
train_data_full = tf.keras.preprocessing.image_dataset_from_directory(
    "Fast Food Classification V2/Train",
    label_mode=LABEL_MODE,
    seed=SEED,  # Any fixed value works for reproducibility
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=None #TODO: Set appropriate value
)

# Load validation dataset
validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    "Fast Food Classification V2/Valid",
    label_mode='int',
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=None #TODO: Set appropriate value
)

# Load the test dataset
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    "Fast Food Classification V2/Test",
    label_mode='int',
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    shuffle=None #TODO: Set appropriate value
)


Found 150 files belonging to 3 classes.
Found 600 files belonging to 3 classes.
Found 4500 files belonging to 3 classes.
Found 1100 files belonging to 3 classes.
Found 400 files belonging to 3 classes.


2. Add normalization and data augmentation as needed to the datasets.  
Which normalization is best? (Tip: Check how the original model was trained.)  
Which data augmentation makes sense?

In [49]:
# Training-specific preprocessing
train_preprocessing = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),  # Normalize pixel values to [0, 1]
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),  # Data augmentation
    tf.keras.layers.RandomRotation(0.2),  # Data augmentation
    tf.keras.layers.RandomZoom(0.2),  # Data augmentation
])

# Validation-specific preprocessing
validation_preprocessing = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),  # Normalize pixel values to [0, 1]
])

# Helper functions to apply preprocessing only to images, not labels
def preprocess_train(image, label):
    return train_preprocessing(image), label

def preprocess_validation(image, label):
    return validation_preprocessing(image), label

# Apply preprocessing to the datasets
train_data_small = train_data_small.map(preprocess_train)
validation_data = validation_data.map(preprocess_validation)

## 3. Model Definitions

Imports and Constants


In [51]:
import tensorflow as tf

CLASSES = ["Taquito", "Crispy Chicken", "Donut"]
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128



### Feature Extraction

1. Create a feature extraction model using the Inception V3 architecture.
2. Freeze all layers of the base model.
3. Add a new classification head with global average pooling, dropout, and a dense layer.


In [55]:
def create_feature_extraction_model():
    """Creates model with frozen base layers (feature extraction)"""
    
    # Load pre-trained InceptionV3 with correct input size
    base_model = tf.keras.applications.InceptionV3(
        weights='imagenet',
        include_top=False,
        input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)
    )

    # Freeze all layers for feature extraction
    base_model.trainable = False

    # Simple classification head
    # - GlobalAveragePooling2D reduces spatial dimensions
    # - Final Dense layer maps to class probabilities
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(CLASSES, activation='softmax')
    ])

    return model

### Fine-Tuning Last Few Layers

1. Create a fine-tuning model using the Inception V3 architecture.
2. Unfreeze the last few layers of the base model.
3. Add a new classification head with global average pooling, dropout, and a dense layer.


In [56]:
def create_fine_tuning_model():
    """Creates model with last few layers unfrozen for fine-tuning"""

    # Load pre-trained InceptionV3 with correct input size
    base_model = tf.keras.applications.InceptionV3(
        weights='imagenet',
        include_top=False,
        input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)
    )

    # Freeze all layers except last few blocks
    base_model.trainable = True
    for layer in base_model.layers[:-20]:  # Unfreeze last 20 layers
        layer.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(CLASSES, activation='softmax')
    ])

    return model


### Full Fine-Tuning

1. Create a fine-tuning model using the Inception V3 architecture.
2. Unfreeze all layers of the base model.
3. Add a new classification head with global average pooling, dropout, and a dense layer.

In [57]:
def create_full_fine_tuning_model():
    """Creates model with all layers unfrozen for full fine-tuning"""

    base_model = tf.keras.applications.InceptionV3(
        weights='imagenet',
        include_top=False,
        input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)
    )

    # Make all layers trainable
    base_model.trainable = True

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(CLASSES, activation='softmax')
    ])

    return model


## 4. Model Training

## Tasks

1. Optimize two different models either using the same transfer learning strategy and two of the three datasets with different sizes or alternatively use one of the datasets with two different learning strategies.

2. Find out what the arguments of the early stopping callback function mean.

3. Choose appropriate learning rates.
Tips for choosing it:
- Start with the Default: Begin with the Adam optimizer's default learning rate of 0.001. This is often a good starting point.
- When fine-tuning (especially full fine-tuning), consider using a smaller learning rate than the default. This helps prevent large updates to the pre-trained weights, which could disrupt the learned features. A learning rate of 1e-4 or 1e-5 is a good starting point.
- The best learning rate will depend on the specific dataset and model. Experiment with different values and observe the training and validation performance. Look for a learning rate that allows the model to converge smoothly without overshooting or getting stuck in a local minimum.



### Example for Training of the Feature Extraction Model with the Small Dataset

In [ ]:
LEARNING_RATE = 0.001

# Settings for the Feature Extraction Model on the small dataset
model_feature_extraction_small = create_feature_extraction_model()
# Compile the model
model_feature_extraction_small.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
# Model Summary
model_feature_extraction_small.summary()


In [71]:
# Define and compile the model if not already done
model_feature_extraction_small = create_feature_extraction_model()
model_feature_extraction_small.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model Training
history_feature_extraction_small = model_feature_extraction_small.fit(
    train_data_small,
    validation_data=validation_data,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 22s 7s/step - accuracy: 0.3108 - loss: 2.9629 - val_accuracy: 0.4036 - val_loss: 1.6723
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/step - accuracy: 0.4630 - loss: 2.0343 - val_accuracy: 0.3473 - val_loss: 2.4803
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/step - accuracy: 0.2434 - loss: 3.3988 - val_accuracy: 0.3536 - val_loss: 2.2839
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 5s/step - accuracy: 0.2796 - loss: 2.8630 - val_accuracy: 0.4382 - val_loss: 1.4680
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 5s/step - accuracy: 0.3179 - loss: 2.2042 - val_accuracy: 0.5418 - val_loss: 1.1730
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 5s/step - accuracy: 0.4362 - loss: 1.6839 - val_accuracy: 0.5309 - val_loss: 1.2317
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 5s/step - accuracy: 0.4636 - loss: 1.5774 - val_accuracy: 0.5682 - val_loss: 1.0899
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 5s/step - accuracy: 0.4867 - loss: 1.6130 - val_accuracy: 0.6082 - val_loss: 0.9892
Epoch 9/